<a href="https://colab.research.google.com/github/SLuis25/Simulaci-n-I/blob/main/Inventario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import random
import pandas as pd

# PARÁMETROS DEL SISTEMA DE INVENTARIO

  # Costos relevantes (según el libro)
costo_orden = 100                      # Costo por cada orden colocada
costo_inventario = 20 / 12            # Costo mensual por unidad en inventario
costo_faltante = 50                   # Costo por unidad no satisfecha

  # Condición inicial del inventario
inventario_inicial = 150


# POLÍTICA DE INVENTARIO (ajustable)

q = 200  # Cantidad a pedir (lote fijo)
R = 100  # Punto de reorden (nivel de inventario que activa el pedido)

# GENERADOR DE DEMANDA SEGÚN TABLA DEL LIBRO

def generar_demanda():
    """
    Devuelve una demanda aleatoria basada en la distribución discreta
    especificada en el Ejemplo 5.5 usando la técnica de transformada inversa.
    """
    r = random.random()
    if r < 0.010: return 35
    elif r < 0.025: return 36
    elif r < 0.045: return 37
    elif r < 0.065: return 38
    elif r < 0.087: return 39
    elif r < 0.110: return 40
    elif r < 0.135: return 41
    elif r < 0.162: return 42
    elif r < 0.190: return 43
    elif r < 0.219: return 44
    elif r < 0.254: return 45
    elif r < 0.299: return 46
    elif r < 0.359: return 47
    elif r < 0.424: return 48
    elif r < 0.494: return 49
    elif r < 0.574: return 50
    elif r < 0.649: return 51
    elif r < 0.719: return 52
    elif r < 0.784: return 53
    elif r < 0.844: return 54
    elif r < 0.894: return 55
    elif r < 0.934: return 56
    elif r < 0.964: return 57
    elif r < 0.980: return 58
    elif r < 0.995: return 59
    else: return 60

# FUNCIÓN DE SIMULACIÓN

def simular_inventario(q, R, meses=12):
    """
    Simula un sistema de inventario con política (q, R) durante un periodo dado.
    Calcula los costos por ordenar, mantener inventario y faltantes.

    Parámetros:
        q (int): Lote de pedido
        R (int): Punto de reorden
        meses (int): Número de meses a simular

    Devuelve:
        DataFrame con resultados mensuales y costo total acumulado
    """
    inventario = inventario_inicial
    pedido_pendiente = False
    tiempo_entrega = 0
    costo_total = 0
    historial = []

    for mes in range(1, meses + 1):
        # Si llega un pedido pendiente este mes
        if tiempo_entrega == 0 and pedido_pendiente:
            inventario += q
            pedido_pendiente = False

        # Generar demanda del mes
        demanda = generar_demanda()
        faltante = max(0, demanda - inventario)
        inventario = max(0, inventario - demanda)

        # Inventario promedio del mes (para calcular costo)
        inventario_prom = (inventario + inventario + demanda) / 2

        # Evaluar si se hace pedido
        orden_colocada = False
        if inventario <= R and not pedido_pendiente:
            tiempo_entrega = random.randint(1, 3)  # Entrega aleatoria de 1 a 3 meses
            pedido_pendiente = True
            orden_colocada = True
            costo_total += costo_orden

        # Cálculo de costos mensuales
        costo_total += inventario_prom * costo_inventario + faltante * costo_faltante

        if pedido_pendiente:
            tiempo_entrega -= 1

        # Guardar resultados del mes
        historial.append({
            "Mes": mes,
            "Demanda": demanda,
            "Inventario Final": inventario,
            "Faltante": faltante,
            "Orden colocada": orden_colocada,
            "Costo acumulado": round(costo_total, 2)
        })

    return pd.DataFrame(historial), round(costo_total, 2)

# EJECUTAR SIMULACIÓN

# Número de meses a simular
n_meses = 30

# Ejecutar
df_resultado, costo_total = simular_inventario(q, R, meses=n_meses)

# Mostrar resultados
print(df_resultado.to_string(index=False))
print(f"\nCosto total simulado para {n_meses} meses: ${costo_total}")

 Mes  Demanda  Inventario Final  Faltante  Orden colocada  Costo acumulado
   1       59                91         0            True           300.83
   2       54               237         0           False           740.83
   3       55               182         0           False          1090.00
   4       38               144         0           False          1361.67
   5       39               105         0           False          1569.17
   6       48                57         0            True          1804.17
   7       47               210         0           False          2193.33
   8       49               161         0           False          2502.50
   9       56               105         0           False          2724.17
  10       49                56         0            True          2958.33
  11       47                 9         0           False          3012.50
  12       52               157         0           False          3317.50
  13       56            